In [60]:
import eventDrivenSimulation as eds
import simulation as sm
import simpy
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import showPaper as sh
import requests
import folium
import haversine
import math
import bibliothek as bib
import random

In [61]:
df=pd.read_csv('ue_data_pandas')
CONFIG = {}

CONFIG['LAT'] = 51.34053598409240
CONFIG['LON'] = 12.381419774766130
CONFIG['RADIUS'] = 500
CONFIG['NUM_UES'] = 1
CONFIG['URL'] = 'https://pqos-api-test.radiolab.dit.htwk-leipzig.de' 

In [121]:
cell_data[20]

{'lon': 12.379552777777777,
 'lat': 51.34256111111111,
 'pci': 133,
 'eci': -1,
 'tac': 32208,
 'az': 180,
 'site_name': 'LY2624',
 'cell_id': 84000256,
 'distance': 259.8570596108029,
 'direction': -23.59291399721252,
 'ue_info': [{'id': 0, 'dl_rx_power': -120, 'ul_rx_power': -120}],
 'served_ues': []}

In [147]:
cell_data[21]

{'lon': 12.377411111111112,
 'lat': 51.34138055555557,
 'pci': 320,
 'eci': -1,
 'tac': 32208,
 'az': 60,
 'site_name': 'LY0494',
 'cell_id': 84000258,
 'distance': 293.8595968675545,
 'direction': -65.993297476071,
 'ue_info': [{'id': 0, 'dl_rx_power': -101, 'ul_rx_power': -101}],
 'served_ues': []}

In [62]:
ul_scenario_map = folium.Map(location = [CONFIG['LAT'], CONFIG['LON']], tiles = "cartodbpositron", zoom_start = 15)

ul_query_string = CONFIG['URL'] + '/generate_scenario' + \
                                  '?lat=' + str(CONFIG['LAT']) + \
                                  '&lon=' + str(CONFIG['LON']) + \
                                  '&radius=' + str(CONFIG['RADIUS']) + \
                                  '&num_ues=' + str(CONFIG['NUM_UES']) + \
                                '&cell_type=NGMN3600'+\
                                '&source=LY_221108'

ul_response_data = requests.get(ul_query_string).json()
ue_data = ul_response_data['ue_data']
cell_data=ul_response_data['cell_data']

eds.plot_map(CONFIG,cell_data)

In [63]:
def create_sector_shape(lon, lat, dir=0, width=120):
    p = [(lat, lon)]
    n_points = 10
    
    for a in range(n_points):
        p.append(haversine.inverse_haversine(p[0], 0.05, (dir - width/2 + width/n_points*a)/180.0 * math.pi))
    
    p.append(p[0])
    return p


In [64]:
c=[319, 134, 774, 318, 776, 775, 132]
df2=df.sample(80)
df_f=df2.groupby('TP1')

ind=np.array([])
ind=np.append(ind,np.array(df_f.get_group(133).index))
ind=np.append(ind,np.array((df_f.get_group(320).index)))

ind2=np.array([])
for i in c:
    ind2=np.append(ind2,np.array(df_f.get_group(i).index))
    
    
df_comp=df.iloc[ind]
df_nocluster=df.iloc[ind2]
df_comp['latitude']=df_comp['lat']
df_comp['longitude']=df_comp['lon']
df_nocluster['latitude']=df_nocluster['lat']
df_nocluster['longitude']=df_nocluster['lon']

#df2.drop(ind)

<ipython-input-64-e7a623a998dd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comp['latitude']=df_comp['lat']
<ipython-input-64-e7a623a998dd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comp['longitude']=df_comp['lon']
<ipython-input-64-e7a623a998dd>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [69]:
df_scenario_15_UEs=pd.read_csv('data_frame_scenario_sinrcluster')
df_scenario_15_UEs['longitude']=df_scenario_15_UEs['lon']
df_scenario_15_UEs['latitude']=df_scenario_15_UEs['lat']


In [70]:
df_comp=df_scenario_15_UEs.copy()

In [71]:
df_stp=pd.DataFrame()
df_mtp=pd.DataFrame()
for i in np.arange(0,len(df_comp)):
    if(df_comp.iloc[i]['SINR-CoMP [dB]']-df_comp.iloc[i]['SINR [dB]']>6 and (df_comp.iloc[i]['TP2'] == 320 or df_comp.iloc[i]['TP2'] == 133)):
        df_mtp=df_mtp.append(df_comp.iloc[i])
    else:
        df_stp=df_stp.append(df_comp.iloc[i])
        

In [95]:
def plot_map_cluster(CONFIG,cell_data,df_r,df_r3):
    ul_scenario_map = folium.Map(location = [CONFIG['LAT'], CONFIG['LON']], tiles  = "cartodbpositron", zoom_start = 15)
    #folium.Circle(radius = CONFIG['RADIUS']-90, 
     #             location = (CONFIG['LAT'], CONFIG['LON']), 
      #            color = 'black', 
       #           fill_color = 'grey',
        #          fill_opacity = 0,
         #         fill = True,
          #        weight = 1,
           #      ).add_to(ul_scenario_map)            

    def plotDotGreen(point):
        folium.Circle(location=[point.latitude, point.longitude],radius=4,weight=5,fill_color = 'black',
                      fill_opacity = 1,fill=True,color='black').add_to(ul_scenario_map)
    #def plotDotRed(point):
     #   folium.Circle(location=[point.latitude, point.longitude],radius=4,weight=5,fill_color = 'grey',
      #                fill_opacity = 1,fill=True,color='grey').add_to(ul_scenario_map)
    def plotDotBlue(point):
        folium.Circle(location=[point.latitude, point.longitude],radius=4,weight=5,fill_color = 'silver',
                      fill_opacity = 1,fill=True,color='silver').add_to(ul_scenario_map)

    df_r.apply(plotDotGreen, axis = 1)
    #df_r2.apply(plotDotRed, axis = 1)
    df_r3.apply(plotDotBlue, axis = 1)


    for cell in cell_data:
        if(cell['pci'] in [320,133]):
            cell_color = 'black'
        else:
            cell_color = 'grey'


        folium.PolyLine(
            create_sector_shape(cell['lon'], cell['lat'], cell['az'], 60), 
            color = cell_color,
            fill_color = cell_color,
           fill_opacity = 0.5, 
            fill = True,
            weight = 2,
            #popup = 'RBs: ' + str(cell['ul_rb_requirement']['mean']),
            tooltip = 'PCI: ' + str(cell['pci'])).add_to(ul_scenario_map)

        folium.Circle(radius = 10, 
                      location = (cell['lat'], cell['lon']), 
                      color = 'black', 
                      fill_color = 'black',
                      fill_opacity = 1,
                      fill = True,
                      weight = 0,
                      popup = cell['site_name']
                     ).add_to(ul_scenario_map)


    
    display(ul_scenario_map)
    ul_scenario_map
    ul_scenario_map.save('mymap.html')
    

In [ ]:
plot_map_cluster(CONFIG,cell_data, df_mtp,df_stp,df_nocluster)

In [90]:
#df_ue=df_mtp[df_mtp['SINR cluster [dB]']-df_mtp['SINR [dB]']>20]

In [98]:
plot_map_cluster(CONFIG,cell_data, df_mtp,df_stp)#,df_nocluster)


In [96]:
plot_map_cluster(CONFIG,cell_data, df_comp,df_nocluster)